# Evaluate the mnist model using 10 png test files
import the needed modules

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import tflite_runtime.interpreter as tflite
import tensorflow as tf

2023-10-11 22:54:38.054454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
interpreter = tflite.Interpreter(model_path="models/number_model_quant.tflite")
interpreter.allocate_tensors() # Needed before execution

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def read_img(img_path):
  """Read MNIST image

  Args:
      img_path (str): path to a MNIST image

  Returns:
      np.array : image in the correct np.array format
  """
  image = Image.open(img_path)
  data = np.asarray(image, dtype=np.float32)
  if data.shape not in [(28, 28), (28, 28, 1)]:
    raise ValueError(
        "Invalid input image shape (MNIST image should have shape 28*28 or 28*28*1)"
    )
  # Normalize the image if necessary
  if data.max() > 1:
    data = data / 255.0
  # Model inference requires batch size one
  data = data.reshape((1, 28, 28))
  return data


In [5]:
input_details = interpreter.get_input_details()[0]
print(input_details)

{'name': 'serving_default_fixed_input:0', 'index': 0, 'shape': array([ 1, 28, 28], dtype=int32), 'shape_signature': array([ 1, 28, 28], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003921568859368563, -128), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [6]:
data_type = input_details["dtype"]
print("data type: ",data_type)
print("index: ",input_details["index"])

data type:  <class 'numpy.int8'>
index:  0


In [7]:
input_quantization_parameters = input_details["quantization_parameters"]
input_scale, input_zero_point = input_quantization_parameters["scales"][0], input_quantization_parameters["zero_points"][0]
print("input scale: {:f}, input_zero_point: {:d}".format(input_scale,input_zero_point))

input scale: 0.003922, input_zero_point: -128


In [8]:
output_details = interpreter.get_output_details()[0]
print(output_details)
print("\nOutput data type: ",output_details["dtype"])
print("index: ",output_details["index"])
print("shape of output tensor: ",output_details['shape'])

{'name': 'StatefulPartitionedCall:0', 'index': 26, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

Output data type:  <class 'numpy.int8'>
index:  26
shape of output tensor:  [ 1 10]


In [9]:
data = read_img("samples/sample8.png")

In [10]:
print("min and max in pixel data: {:f}, {:f}".format(data.min(),data.max()))

min and max in pixel data: 0.000000, 1.000000


In [11]:
data = (data / input_scale + input_zero_point).astype(data_type)
print("min and max in pixel data after quantization: {:f}, {:f}",data.min(),data.max())

min and max in pixel data after quantization: {:f}, {:f} -128 126


In [12]:
interpreter.set_tensor(input_details['index'],data)

In [13]:
interpreter.invoke()
output_data = interpreter.get_tensor(output_details['index']).reshape(10,)
print("shape of output_data :",output_data.shape)
print(output_data, type(output_data),output_data.dtype)

shape of output_data : (10,)
[-128 -128 -128 -128 -128 -128 -128 -128  127 -128] <class 'numpy.ndarray'> int8


In [14]:
digit = np.argmax(output_data)
print("predicted digit: ",digit)
print(output_data)
probs = [None]*10
print("probabilites: ")
for i in range(10):
    probs[i] = (output_data[i] + 128) / 255
    print("{:6.4f}".format(probs[i]), end=" ")
print("")

predicted digit:  8
[-128 -128 -128 -128 -128 -128 -128 -128  127 -128]
probabilites: 
0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 1.0000 0.0000 


To get the probabilities we have to dequantize the result

In [15]:
# dequantize the result
output_quantization_parameters = output_details["quantization_parameters"]
output_scale, output_zero_point = output_quantization_parameters["scales"][0], output_quantization_parameters["zero_points"][0]
print(output_scale * (output_data.astype("float") - output_zero_point))

[0.         0.         0.         0.         0.         0.
 0.         0.         0.99609375 0.        ]


Now that we know how to invoke the interpreter for a single image and how to interpret the result, we can do this for all
10 sample images and calculate the confusion matrix

In [16]:
images = [None] * 10
probabilities = [None] * 10
raw_output_data = [None] * 10
input_details = interpreter.get_input_details()[0]
print("input data type: ", input_details["dtype"])
# get the input quantization parameters and quantize the pixel values
input_quantization_parameters = input_details["quantization_parameters"]
input_scale, input_zero_point = input_quantization_parameters["scales"][0], input_quantization_parameters["zero_points"][0]
print("input scale: {:f}, input_zero_point: {:d}".format(input_scale,input_zero_point))
output_quantization_parameters = output_details["quantization_parameters"]
output_scale, output_zero_point = output_quantization_parameters["scales"][0], output_quantization_parameters["zero_points"][0]
print("output scale: {:f}, output_zero_point: {:d}".format(output_scale,output_zero_point))

for img in range(10):
    # construct the filename
    filename = "samples/sample{:d}".format(img) + ".png"
    images[img] = read_img(filename)
    if img == 0:
        print("data type of pixels: ",images[img].dtype)
    # quantize the image data
    images[img] = (images[img] / input_scale + input_zero_point).astype(data_type)
    if img == 0:
        print("Data type of pixels passed to the interpreter: ",images[img].dtype)
    if img == 0:
        print("min and max pixel values after quantization: {:d},{:d}".format(images[img].min(),images[img].max()))
    # set the input tensor
    interpreter.set_tensor(input_details['index'],images[img])
    # and invoke inference
    interpreter.invoke()
    output = interpreter.get_tensor(output_details['index']).reshape(10,)
    #if img == 0:
    print(output)
    # dequantize to get the probabilities
    probabilities[img] = output_scale * (output.astype("float") - output_zero_point)
    raw_output_data[img] = output

input data type:  <class 'numpy.int8'>
input scale: 0.003922, input_zero_point: -128
output scale: 0.003906, output_zero_point: -128
data type of pixels:  float32
Data type of pixels passed to the interpreter:  int8
min and max pixel values after quantization: -128,126
[ 127 -128 -128 -128 -128 -128 -128 -128 -128 -128]
[-128  127 -128 -128 -128 -128 -128 -128 -128 -128]
[-128 -128  127 -128 -128 -128 -128 -128 -128 -128]
[-128 -128 -123  119 -128 -128 -128 -128 -125 -128]
[-128 -128 -128 -128  127 -128 -128 -128 -128 -128]
[-128 -128 -128 -128 -128  127 -128 -128 -128 -128]
[-126 -128 -128 -128 -128 -128  126 -128 -128 -128]
[-128 -128 -128 -128 -128 -128 -128  127 -128 -128]
[-128 -128 -128 -128 -128 -128 -128 -128  127 -128]
[-128 -128 -127 -125 -128 -128 -128 -128 -128  124]


In [17]:
print("The confusion matrix:")
for i in range(10):
    for j in range(10):
        print("{:8.6f}".format(probabilities[i][j]), end=" ")
    print("")

The confusion matrix:
0.996094 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.996094 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 0.996094 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 0.019531 0.964844 0.000000 0.000000 0.000000 0.000000 0.011719 0.000000 
0.000000 0.000000 0.000000 0.000000 0.996094 0.000000 0.000000 0.000000 0.000000 0.000000 
0.000000 0.000000 0.000000 0.000000 0.000000 0.996094 0.000000 0.000000 0.000000 0.000000 
0.007812 0.000000 0.000000 0.000000 0.000000 0.000000 0.992188 0.000000 0.000000 0.000000 
0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.996094 0.000000 0.000000 
0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.996094 0.000000 
0.000000 0.000000 0.003906 0.011719 0.000000 0.000000 0.000000 0.000000 0.000000 0.984375 


In [18]:
for i in range(10):
    print("Digits predicted: {:d}".format(probabilities[i].argmax()))

Digits predicted: 0
Digits predicted: 1
Digits predicted: 2
Digits predicted: 3
Digits predicted: 4
Digits predicted: 5
Digits predicted: 6
Digits predicted: 7
Digits predicted: 8
Digits predicted: 9


Try to improve probabilities

In [ ]:
probs = [None]*10
for img in range(10):
    probs[img] = raw_output_data[img].astype(float)
    for i in range(10):        
        probs[img][i] = (raw_output_data[img][i] + 128) / 255
        print("{:8.6f}".format(probs[img][i]),end=" ")
    print("")

check if probabilities sum to 1.0

In [ ]:
for img in range(10):
    sum = 0
    for i in range(10):
        sum += probabilities[img][i]
    print("sum of probabilities for digit {:d}: {:8.2f}".format(img,sum))

In [ ]:
for img in range(10):
    sum = 0
    for i in range(10):
        sum += probs[img][i]
    print("sum of probabilities for digit {:d} for custom dequantization: {:8.2f}".format(img,sum))

Plot the confusion matrix

In [ ]:
from matplotlib import colors as colors
plt.matshow(probs,norm=colors.LogNorm(0,1), cmap="viridis")